# Geth Docker Setup
For Geth, we will have to run this script first then run docker compose file later.


In [ ]:
import os

path = os.path.abspath("")
print(path)

store_path = os.path.join(path, "geth/")
print(store_path)

env_path = os.path.abspath(os.path.join(path, ".."))
print(env_path)

In [ ]:
# Get "GETH_NETWORK_ID" and "GETH_NETWORK_NAME" from .env file
import dotenv

env_file = os.path.join(env_path, ".env")
print(env_file)

dotenv.load_dotenv(env_file)

network_id = os.getenv("GETH_NETWORK_ID")
network_acc = os.getenv("GETH_ACCOUNT_PASSWORD")

if network_id is None or network_acc is None:
    raise Exception("GETH_NETWORK_ID or GETH_NETWORK_NAME is not set")

print(network_id)
print(network_acc)

In [ ]:
# Copy to .env inside store_path (create if not exists)
import shutil

store_env_file = os.path.join(store_path, ".env")
print(store_env_file)

shutil.copy(env_file, store_env_file)

## This one is command after running the script

And then you decrypt the file using password defined in the .env file.

In [ ]:
# docker cp mycontainer:/root/.ethereum/keystore ./keystore
container_name = "geth-geth-bootnode-1"
container_keystore = "/root/.ethereum/keystore"

os.system(f"docker cp {container_name}:{container_keystore} {store_path}")

In [ ]:
# check if store_path/keystore exists, else skip this step
import json
import web3

keystore_path = os.path.join(store_path, "keystore")

if not os.path.exists(keystore_path):
    raise Exception("Keystore path does not exist")

keystore_files = os.listdir(keystore_path)

if len(keystore_files) > 1:
    raise Exception("More than 1 keystore file found")

keystore_file = keystore_files[0]
with open(os.path.join(keystore_path, keystore_file)) as f:
    keystore = json.load(f)
    print(f"Public address: 0x{keystore['address']}")
    password = os.getenv("GETH_ACCOUNT_PASSWORD")
    private_key = web3.Account.decrypt(keystore, password)
    print(f"Private key: {private_key.hex()}")

In [ ]:
# Read .env file and split into lines
with open(env_file, "r") as f:
    lines = f.readlines()

# Create a dictionary of existing environment variables
env_vars = dict(line.strip().split("=", 1) for line in lines if line.strip() and not line.strip().startswith("#"))

# Update the values
env_vars["GETH_ACCOUNT_PUBLIC_KEY"] = f'0x{keystore["address"]}'
env_vars["GETH_ACCOUNT_PRIVATE_KEY"] = private_key.hex()

# Write the updated environment variables back to the .env file
with open(env_file, "w") as f:
    f.writelines(f"{k}={v}\n" for k, v in env_vars.items())